In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

import tempfile

import tensorflow.compat.v1 as tf

from tensorflow import keras
print("TensorFlow version is ", tf.__version__)

import numpy as np

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

TensorFlow version is  2.1.0


In [3]:
base_dir = os.getcwd()
train_dir = os.path.join(base_dir, 'train_images_pre')
dev_dir = os.path.join(base_dir, 'dev_images_pre')
test_dir = os.path.join(base_dir, 'test_images_pre')

train_0_dir = os.path.join(train_dir, '0')
print ('Total training class 0 images:', len(os.listdir(train_0_dir)))

train_1_dir = os.path.join(train_dir, '1')
print ('Total training class 1 images:', len(os.listdir(train_1_dir)))

train_2_dir = os.path.join(train_dir, '2')
print ('Total training class 2 images:', len(os.listdir(train_2_dir)))

train_3_dir = os.path.join(train_dir, '3')
print ('Total training class 3 images:', len(os.listdir(train_3_dir)))

train_4_dir = os.path.join(train_dir, '4')
print ('Total training class 4 images:', len(os.listdir(train_4_dir)))

Total training class 0 images: 1264
Total training class 1 images: 259
Total training class 2 images: 699
Total training class 3 images: 135
Total training class 4 images: 207


In [4]:
dev_0_dir = os.path.join(dev_dir, '0')
print ('Total dev class 0 images:', len(os.listdir(dev_0_dir)))

dev_1_dir = os.path.join(dev_dir, '1')
print ('Total dev class 1 images:', len(os.listdir(dev_1_dir)))

dev_2_dir = os.path.join(dev_dir, '2')
print ('Total dev class 2 images:', len(os.listdir(dev_2_dir)))

dev_3_dir = os.path.join(dev_dir, '3')
print ('Total dev class 3 images:', len(os.listdir(dev_3_dir)))

dev_4_dir = os.path.join(dev_dir, '4')
print ('Total dev class 4 images:', len(os.listdir(dev_4_dir)))

Total dev class 0 images: 271
Total dev class 1 images: 56
Total dev class 2 images: 150
Total dev class 3 images: 29
Total dev class 4 images: 44


In [5]:
test_0_dir = os.path.join(test_dir, '0')
print ('Total test class 0 images:', len(os.listdir(test_0_dir)))

test_1_dir = os.path.join(test_dir, '1')
print ('Total test class 1 images:', len(os.listdir(test_1_dir)))

test_2_dir = os.path.join(test_dir, '2')
print ('Total test class 2 images:', len(os.listdir(test_2_dir)))

test_3_dir = os.path.join(test_dir, '3')
print ('Total test class 3 images:', len(os.listdir(test_3_dir)))

test_4_dir = os.path.join(test_dir, '4')
print ('Total test class 4 images:', len(os.listdir(test_4_dir)))

Total test class 0 images: 270
Total test class 1 images: 55
Total test class 2 images: 150
Total test class 3 images: 29
Total test class 4 images: 44


In [6]:
image_size = 224
batch_size = 32

# Rescale all images by 1./255 and apply image augmentation
train_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255,rotation_range=360, horizontal_flip=True,vertical_flip=True)

dev_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

test_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Flow training images in batches using train_datagen generator
train_generator = train_datagen.flow_from_directory(
                train_dir,  # Source directory for the training images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                # Since we use binary_crossentropy loss, we need binary labels
                class_mode='categorical')

# Flow dev images in batches using dev_datagen generator
dev_generator = dev_datagen.flow_from_directory(
                dev_dir, # Source directory for the dev images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                class_mode='categorical')

# Flow test images in batches using test_datagen generator
test_generator = test_datagen.flow_from_directory(
                test_dir, # Source directory for the test images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                class_mode='categorical')

Found 2564 images belonging to 5 classes.
Found 550 images belonging to 5 classes.
Found 548 images belonging to 5 classes.


In [7]:
IMG_SHAPE = (image_size, image_size, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')
base_model.trainable = True
base_model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128    

In [8]:
# Add fully connected layers
model = tf.keras.Sequential([
  base_model,
  keras.layers.GlobalAveragePooling2D(),
  keras.layers.Dense(5, activation='softmax')
])

In [9]:
# Initialize weights 
model.load_weights("Reg01/cp.ckpt")

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])

model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Model) (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 5)                 6405      
Total params: 2,264,389
Trainable params: 2,230,277
Non-trainable params: 34,112
_________________________________________________________________


In [10]:
#set up callback
checkpoint_path = "Reg01FT/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [11]:
#train model
epochs = 10
steps_per_epoch = train_generator.n // batch_size
dev_steps = dev_generator.n // batch_size

history = model.fit_generator(train_generator,
                              steps_per_epoch = steps_per_epoch,
                              epochs=epochs,
                              workers=4,
                              validation_data=dev_generator,
                              validation_steps=dev_steps, callbacks=[cp_callback],
                              class_weight={0:641./1264., 1:641./259., 2:641./699., 3:641./135., 4:641./207.})

Epoch 1/10
79/80 [============================>.] - ETA: 7s - loss: 1.6248 - categorical_accuracy: 0.5920 
Epoch 00001: saving model to Reg01FT/cp.ckpt
80/80 [==============================] - 636s 8s/step - loss: 1.6155 - categorical_accuracy: 0.5940 - val_loss: 2.5604 - val_categorical_accuracy: 0.5184
Epoch 2/10
79/80 [============================>.] - ETA: 7s - loss: 1.3425 - categorical_accuracy: 0.6780 
Epoch 00002: saving model to Reg01FT/cp.ckpt
80/80 [==============================] - 615s 8s/step - loss: 1.3507 - categorical_accuracy: 0.6801 - val_loss: 4.7208 - val_categorical_accuracy: 0.5018
Epoch 3/10
79/80 [============================>.] - ETA: 7s - loss: 1.1984 - categorical_accuracy: 0.7292 
Epoch 00003: saving model to Reg01FT/cp.ckpt
80/80 [==============================] - 612s 8s/step - loss: 1.1998 - categorical_accuracy: 0.7295 - val_loss: 4.8346 - val_categorical_accuracy: 0.4945
Epoch 4/10
 5/80 [>.............................] - ETA: 12:40 - loss: 1.4269 - ca

KeyboardInterrupt: 

In [ ]:
#plot accuracy and loss
acc = history.history['categorical_accuracy']
val_acc = history.history['val_categorical_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,max(plt.ylim())])
plt.title('Training and Validation Loss')
plt.savefig('attempt1_plots.png')
plt.show()